In [1]:
from time import time
from bs4 import BeautifulSoup
from numpy.lib import utils
import requests
import os
import glob
import pandas as pd
from datetime import datetime
import utils
import datetime

In [ ]:
# Concat all file xlsx to summary link of News
# cwd = os.path.abspath('/Users/hieudt/VNU/Thesis/Vnexpress/2020-2023') 
# files = os.listdir(cwd)

# df = pd.DataFrame()
# for file in files:
#      if file.endswith('.xlsx'):
#          df = df.append(pd.read_excel(file), ignore_index=True) 
# df = df[['web-scraper-start-url','news','news-href']].drop_duplicates()
# df.to_excel('/Users/hieudt/VNU/Thesis/Vnexpress/2020-2023/news_vnexpress.xlsx', index=False)

In [120]:
# Function read content from link
def read_content(link):
    content_list = []
    response = requests.get(link)   
    bts = BeautifulSoup(response.content, "html.parser")
    
    content = bts.find_all('p')
    for i in content:
        content_list.append(i.text)
    
    time = bts.find('span', attrs={"class":"date"})
    if time is None:
        time = "Not defined"
    else:
        time = time.text
    
    location = bts.find('span', attrs={"class":"location-stamp"})
    if location is None:
        location = "Not defined"
    else:
        location = location.text.replace('\r\n', '')
    return ' '.join(content_list), time, location

In [168]:
df = links[['news','news-href']]
df['content'] = ''
df['time'] = ''
df['location'] = ''

In [171]:
for i in range(len(df)):
    print('------------reading content sequence',i)
    df['content'][i] = read_content(df['news-href'][i])[0]
    df['time'][i] = read_content(df['news-href'][i])[1]
    df['location'][i] = read_content(df['news-href'][i])[2]

------------reading content sequence 0
------------reading content sequence 1
------------reading content sequence 2
------------reading content sequence 3
------------reading content sequence 4
------------reading content sequence 5
------------reading content sequence 6
------------reading content sequence 7
------------reading content sequence 8
------------reading content sequence 9
------------reading content sequence 10
------------reading content sequence 11
------------reading content sequence 12
------------reading content sequence 13
------------reading content sequence 14
------------reading content sequence 15
------------reading content sequence 16
------------reading content sequence 17
------------reading content sequence 18
------------reading content sequence 19
------------reading content sequence 20
------------reading content sequence 21
------------reading content sequence 22
------------reading content sequence 23
------------reading content sequence 24
----------

In [175]:
# df.to_excel('/Users/hieudt/VNU/Thesis/Vnexpress/2020-2023/vnexpress_2020_2023.xlsx', index=False)

In [176]:
df = pd.read_csv('/Users/hieudt/VNU/Thesis/Vnexpress/2020-2023/vnexpress_2020_2023.csv')

In [183]:
def convert_date(date):
    '''
        convert string to date: dd/mm/yyyy
    '''
    date = date.split('/')
    year = int(date[2])
    month = int(date[1])
    day = int(date[0])
    return datetime.datetime(year, month, day), pd.Timestamp(datetime.datetime(year, month, day)).quarter, year, month

def base_index(list_of_group_words, text):
    count = 0
    for i in list_of_group_words:
        if is_subset(i, text):
            count += 1
    return count

def location_process(text):
    text = text.strip()
    text = text.replace(':', '')
    return text

def time_process(text):
    if text != 'Not defined':
        text = text.split(',')[1].split('-')[0]
        text = convert_date(text)
    return text

In [184]:
def is_subset(group_words, text):
    """
        Check whether a word combination is in a text or not
        group_words: list of words group need to check
        text: content of a paper
    """
    status = None
    for i in group_words:
        if i.upper() in text.upper():
            status = True
        else:
            status = False
            break
    return status


def base_index(list_of_group_words, text):
    count = 0
    list_out = []
    for i in list_of_group_words:
        if is_subset(i,text):
            list_out.append(i)
            count +=1
    return count,list_out


def word_combination(excel_file_link):
    word_combination = pd.read_excel(excel_file_link)
    word_combination['list'] = word_combination.apply(lambda row: row['group_words'].split(','), axis=1)
    list_of_group_words = []
    for i in word_combination['list']:
        list_of_group_words.append(i)
    return list_of_group_words

In [193]:
def vnexpress_all_processing():
    vnexpress = pd.read_excel('/Users/hieudt/VNU/Thesis/Vnexpress/2020-2023/vnexpress_2020_2023.xlsx')
    vnexpress = vnexpress.dropna()
    vnexpress['location'] = vnexpress.apply(lambda row: location_process(row['location']), axis=1)
    vnexpress['Month'] = vnexpress.apply(lambda row: time_process(row['time'])[3], axis=1)
    vnexpress['Quarter'] = vnexpress.apply(lambda row: time_process(row['time'])[1], axis=1)
    vnexpress['Year'] = vnexpress.apply(lambda row: time_process(row['time'])[2], axis=1)
    vnexpress.to_excel('/Users/hieudt/VNU/Thesis/Vnexpress/2020-2023/vnexpress_all_processing.xlsx', index=False, encoding='utf-8-sig')


def vnexpress_result():
    list_of_group_words = word_combination('/Users/hieudt/VNU/Thesis/Thesis-Data_Science-main/word_combination.xlsx')
    data = pd.read_excel('/Users/hieudt/VNU/Thesis/Vnexpress/2020-2023/vnexpress_all_processing.xlsx')
    data['based_index'] = data.apply(lambda row: base_index(list_of_group_words, row['content'])[0], axis=1)
    data['List_out'] = data.apply(lambda row: base_index(list_of_group_words, row['content'])[1], axis=1)
    data = data[data['based_index'] > 0]
    data.to_excel('/Users/hieudt/VNU/Thesis/Vnexpress/2020-2023/vnexpress_result.xlsx', index=False, encoding='utf-8-sig')

In [191]:
vnexpress_all_processing()

In [194]:
vnexpress_result()